In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization, concatenate, AveragePooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
from tqdm import tqdm



In [2]:
data = pd.read_csv('/kaggle/input/groundtable/ISIC_2019_Training_GroundTruth.csv')
data.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
store_list = []
image_height = 224
image_width = 224
for i in tqdm(range(data.shape[0])):
    path = '/kaggle/input/isicdataset2019/ISIC_2019_Training_Input/' + data['image'][i] + '.jpg'
    image_check = image.load_img(path, target_size=(image_height, image_width))
    image_check = image.img_to_array(image_check)
    # scaling the images
    image_check = image_check/255
    store_list.append(image_check)

x = np.array(store_list)
y = data.drop(columns=['image'])
Y = y.to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x, Y, random_state=42, test_size=0.20)

 86%|████████▌ | 857/1000 [00:12<00:01, 77.09it/s]

In [ ]:
# Define the model architecture
input_layer = keras.layers.Input(shape=x_train[0].shape)

# First Inception module
conv1 = Conv2D(64, (1,1), padding='same', activation='relu')(input_layer)
conv2 = Conv2D(96, (1,1), padding='same', activation='relu')(input_layer)
conv2 = Conv2D(128, (3,3), padding='same', activation='relu')(conv2)
conv3 = Conv2D(16, (1,1), padding='same', activation='relu')(input_layer)
conv3 = Conv2D(32, (5,5), padding='same', activation='relu')(conv3)
pool = MaxPool2D((3,3), strides=(1,1), padding='same')(input_layer)
pool = Conv2D(32, (1,1), padding='same', activation='relu')(pool)
output1 = concatenate([conv1, conv2, conv3, pool], axis=-1)

# Second Inception module
conv1 = Conv2D(128, (1,1), padding='same', activation='relu')(output1)
conv2 = Conv2D(128, (1,1), padding='same', activation='relu')(output1)
conv2 = Conv2D(192, (3,3), padding='same', activation='relu')(conv2)
conv3 = Conv2D(32, (1,1), padding='same', activation='relu')(output1)
conv3 = Conv2D(96, (5,5), padding='same', activation='relu')(conv3)
pool = MaxPool2D((3,3), strides=(1,1), padding='same')(output1)
pool = Conv2D(64, (1,1), padding='same', activation='relu')(pool)
output2 = concatenate([conv1, conv2, conv3, pool], axis=-1)

# Third Inception module
conv1 = Conv2D(192, (1,1), padding='same', activation='relu')(output2)
conv2 = Conv2D(96, (1,1), padding='same', activation='relu')(output2)
conv2 = Conv2D(208, (3,3), padding='same', activation='relu')(conv2)
conv3 = Conv2D(16, (1,1), padding='same', activation='relu')(output2)
conv3 = Conv2D(48, (5,5), padding='same', activation='relu')(conv3)
pool = MaxPool2D((3,3), strides=(1,1), padding='same')(output2)
pool = Conv2D(64, (1,1), padding='same', activation='relu')(pool)
output3 = concatenate([conv1, conv2, conv3, pool], axis=-1)

# Fourth Inception module
conv1 = Conv2D(160, (1,1), padding='same', activation='relu')(output3)
conv2 = Conv2D(112, (1,1), padding='same', activation='relu')(output3)
conv2 = Conv2D(224, (3,3), padding='same', activation='relu')(conv2)
conv3 = Conv2D(24, (1,1), padding='same', activation='relu')(output3)
conv3 = Conv2D(64, (5,5), padding='same', activation='relu')(conv3)
pool = MaxPool2D((3,3), strides=(1,1), padding='same')(output3)
pool = Conv2D(64, (1,1), padding='same', activation='relu')(pool)
output4 = concatenate([conv1, conv2, conv3, pool], axis=-1)

# Fifth Inception module
conv1 = Conv2D(128, (1,1), padding='same', activation='relu')(output4)
conv2 = Conv2D(128, (1,1), padding='same', activation='relu')(output4)
conv2 = Conv2D(256, (3,3), padding='same', activation='relu')(conv2)
conv3 = Conv2D(24, (1,1), padding='same', activation='relu')(output4)
conv3 = Conv2D(64, (5,5), padding='same', activation='relu')(conv3)
pool = MaxPool2D((3,3), strides=(1,1), padding='same')(output4)
pool = Conv2D(64, (1,1), padding='same', activation='relu')(pool)
output5 = concatenate([conv1, conv2, conv3, pool], axis=-1)

# Average pooling and output layer
pool = AveragePooling2D((7,7))(output5)
flatten = Flatten()(pool)
output_layer = Dense(9, activation='sigmoid')(flatten)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)
model.summary()


In [ ]:
model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
es_callback = keras.callbacks.EarlyStopping(patience=3)
history = model.fit(x_train, y_train, epochs=15, validation_split=0.3, callbacks=[es_callback])

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Make predictions on the test set
y_pred = model.predict(x_test)

# Print the actual and predicted data side by side
for i in range(len(y_test)):
    print('Actual:', y_test[i], 'Predicted:', y_pred[i])

# Plot the accuracy and loss graphs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()